##Importing dependencies

In [ ]:
import pandas as pd
import numpy as np
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from sklearn.tree import DecisionTreeClassifier

In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Add the path of the dataset file in the following line
path = "/content/drive/MyDrive/Machine learning/Fake News"


tdf = pd.read_csv("/content/drive/MyDrive/Machine learning/Fake News/train.csv")

Mounted at /content/drive


In [ ]:
tdf.shape

(20800, 5)

In [ ]:
tdf.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [ ]:
tdf.tail()

,id,title,author,text,label
20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...,0
20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...,0
20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...,0
20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal...",1
20799,20799,What Keeps the F-35 Alive,David Swanson,"David Swanson is an author, activist, journa...",1


In [ ]:
tdf.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [ ]:
tdf=tdf.fillna('')

In [ ]:
tdf['content']=tdf['author'] +' '+ tdf['title']+tdf['text']

In [ ]:
print(tdf['content'])

0        Darrell Lucus House Dem Aide: We Didn’t Even S...
1        Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo...
2        Consortiumnews.com Why the Truth Might Get You...
3        Jessica Purkiss 15 Civilians Killed In Single ...
4        Howard Portnoy Iranian woman jailed for fictio...
                               ...                        
20795    Jerome Hudson Rapper T.I.: Trump a ’Poster Chi...
20796    Benjamin Hoffman N.F.L. Playoffs: Schedule, Ma...
20797    Michael J. de la Merced and Rachel Abrams Macy...
20798    Alex Ansary NATO, Russia To Hold Parallel Exer...
20799    David Swanson What Keeps the F-35 Alive  David...
Name: content, Length: 20800, dtype: object


In [ ]:
X=tdf.drop(columns='label',axis=1)  #axis=0 is for row
Y=tdf['label']

In [ ]:
print(X)
print(Y)

          id                                              title  \
0          0  House Dem Aide: We Didn’t Even See Comey’s Let...   
1          1  FLYNN: Hillary Clinton, Big Woman on Campus - ...   
2          2                  Why the Truth Might Get You Fired   
3          3  15 Civilians Killed In Single US Airstrike Hav...   
4          4  Iranian woman jailed for fictional unpublished...   
...      ...                                                ...   
20795  20795  Rapper T.I.: Trump a ’Poster Child For White S...   
20796  20796  N.F.L. Playoffs: Schedule, Matchups and Odds -...   
20797  20797  Macy’s Is Said to Receive Takeover Approach by...   
20798  20798  NATO, Russia To Hold Parallel Exercises In Bal...   
20799  20799                          What Keeps the F-35 Alive   

                                          author  \
0                                  Darrell Lucus   
1                                Daniel J. Flynn   
2                             Consortiu

###STEMMING OF TEXT DATA

In [ ]:
port_stem = PorterStemmer()

In [ ]:
def stemming(content):
  stemmed_content = re.sub('[^a-zA-Z]',' ',content)
  stemmed_content = stemmed_content.lower()
  stemmed_content = stemmed_content.split()
  # for word in stemmed_content:
  #   if not word in stopwords.words('english'):
  #     stemmed_content = port_stem.stem(word)
  stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
  stemmed_content = ' '.join(stemmed_content)
  return stemmed_content

In [ ]:
tdf['content'] = tdf['content'].apply(stemming)

In [ ]:
# print(tdf['content'].iloc[21:31])
# tdf['content'][21:31]
tdf[3:4][2:3]

,id,title,author,text,label,content


In [ ]:
tdf

,id,title,author,text,label,content
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1,darrel lucu hous dem aid even see comey letter...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0,daniel j flynn flynn hillari clinton big woman...
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1,consortiumnew com truth might get firedwhi tru...
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1,jessica purkiss civilian kill singl us airstri...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1,howard portnoy iranian woman jail fiction unpu...
...,...,...,...,...,...,...
20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...,0,jerom hudson rapper trump poster child white s...
20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...,0,benjamin hoffman n f l playoff schedul matchup...
20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...,0,michael j de la merc rachel abram maci said re...
20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal...",1,alex ansari nato russia hold parallel exercis ...


In [ ]:
X = tdf['content'].values
Y = tdf['label'].values
X2 = X
Y2 = Y

In [ ]:
X = X.astype(str)
print(X.dtype)
print(Y)
X

<U72057
[1 0 1 ... 0 1 1]


array(['darrel lucu hous dem aid even see comey letter jason chaffetz tweet ithous dem aid even see comey letter jason chaffetz tweet darrel lucu octob subscrib jason chaffetz stump american fork utah imag courtesi michael jolley avail creativ common licens apolog keith olbermann doubt worst person world week fbi director jame comey accord hous democrat aid look like also know second worst person well turn comey sent infam letter announc fbi look email may relat hillari clinton email server rank democrat relev committe hear comey found via tweet one republican committe chairmen know comey notifi republican chairmen democrat rank member hous intellig judiciari oversight committe agenc review email recent discov order see contain classifi inform long letter went oversight committe chairman jason chaffetz set polit world ablaz tweet fbi dir inform fbi learn exist email appear pertin investig case reopen jason chaffetz jasoninthehous octob cours know case comey actual say review email ligh

### text to numerical data : Vectorizing

In [ ]:
vectorizer = TfidfVectorizer()       #from scikit-learn library
vectorizer.fit(X)
X = vectorizer.transform(X)

In [ ]:
print(X.shape)

(20800, 120122)


In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, stratify=Y, random_state=30)

## applying Logistic Regression

In [ ]:
model = LogisticRegression()

In [ ]:
model.fit( X_train,Y_train)

LogisticRegression()

In [ ]:
X_train_prediction = model.predict(X_train)
training_data_accuracy = accuracy_score(X_train_prediction,Y_train)
print("accuracy_score of training data",training_data_accuracy)
print(classification_report(X_train_prediction,Y_train))

accuracy_score of training data 0.9745793269230769
              precision    recall  f1-score   support

           0       0.97      0.98      0.97      8273
           1       0.98      0.97      0.97      8367

    accuracy                           0.97     16640
   macro avg       0.97      0.97      0.97     16640
weighted avg       0.97      0.97      0.97     16640



In [ ]:
X_test_prediction=model.predict(X_test)
test_data_accuracy=accuracy_score(X_test_prediction,Y_test)
print("accuracy_score of test data",test_data_accuracy)
print(classification_report(X_test_prediction,Y_test))

accuracy_score of test data 0.9560096153846154
              precision    recall  f1-score   support

           0       0.95      0.96      0.96      2068
           1       0.96      0.95      0.96      2092

    accuracy                           0.96      4160
   macro avg       0.96      0.96      0.96      4160
weighted avg       0.96      0.96      0.96      4160



In [ ]:
X[0].dtype

dtype('float64')

## applying Decision Tree

In [ ]:
X_train1, X_test1, Y_train1, Y_test1 = train_test_split(X2,Y2, test_size=0.2,stratify=Y, random_state=30)

In [ ]:
X_train1 = X_train1.astype(str)
X_test1 = X_test1.astype(str)

tfidf_vectorizer = TfidfVectorizer(stop_words='english', max_df=0.7)
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train1)
X_test_tfidf = tfidf_vectorizer.transform(X_test1)


In [ ]:
X_test_tfidf.shape

(4160, 106625)

In [ ]:
# Define the decision tree classifier model
dt = DecisionTreeClassifier(random_state=30)

In [ ]:
# Define the hyperparameters to tune using GridSearchCV
params = {
    'max_depth': [5, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

In [ ]:
# Use GridSearchCV to find the best hyperparameters
grid_search = GridSearchCV(estimator=dt, param_grid=params, cv=5)
grid_search.fit(X_train_tfidf, Y_train1)

GridSearchCV(cv=5, estimator=DecisionTreeClassifier(random_state=30),
             param_grid={'max_depth': [5, 10, 20],
                         'min_samples_leaf': [1, 2, 4],
                         'min_samples_split': [2, 5, 10]})

In [ ]:
# Train the decision tree model with the best hyperparameters
dt = DecisionTreeClassifier(**grid_search.best_params_, random_state=30)
dt.fit(X_train_tfidf, Y_train1)

DecisionTreeClassifier(max_depth=20, min_samples_leaf=2, min_samples_split=5,
                       random_state=30)

In [ ]:
# Evaluate the model on the test set
y_pred = dt.predict(X_test_tfidf)
accuracy = accuracy_score(Y_test1, y_pred)
print('Accuracy:', accuracy)
print(classification_report(Y_test1, y_pred))

Accuracy: 0.9382211538461539
              precision    recall  f1-score   support

           0       0.94      0.93      0.94      2077
           1       0.93      0.95      0.94      2083

    accuracy                           0.94      4160
   macro avg       0.94      0.94      0.94      4160
weighted avg       0.94      0.94      0.94      4160

